In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

%matplotlib inline
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LassoCV, Lasso, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from sklearn.model_selection import train_test_split
import math
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error, median_absolute_error
import xgboost as xgb
import datetime as datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.cross_validation import KFold
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
station = pd.read_csv('../input/station.csv')
station.sort_values(by = ['city'])
trip = pd.read_csv('../input/trip.csv', parse_dates=['start_date'])
station_city = station[['id','city']]


In [ ]:
def get_city_data(tripc,city):
    trip = tripc[tripc['city']==city]
    return trip
## data merge
tripc = trip.merge(station_city,left_on = 'start_station_id',right_on = 'id',how = 'inner')
#extract the trip information for the city San Jose
city = 'San Jose'
trip_SJ = get_city_data(tripc,city)



In [ ]:
def rmse(y_hat,y):
    diff = y_hat - y
    nrerr = np.mean(diff*diff)
    err = np.sqrt(nrerr)
    return err
def get_label(trip):
#Baseline model: use the mean value as the prediction result and calculate the rmse
    trip['date'] = trip.start_date.dt.date
#count how many trips occured on a day
    dates = {}
    for d in trip.date:
        if d not in dates:
            dates[d] = 1
        if d in dates:
            dates[d] = dates[d] + 1
#extract the label (daily trip number and corresponding day)
    trip2 = pd.DataFrame.from_dict(dates,orient = "index")
    trip2['date'] = trip2.index
    trip2['trips'] = trip2.iloc[:,0]
    train = trip2.iloc[:,1:3]
    train.reset_index(drop = True, inplace = True)
    return train
def base_model(trip):
    #split the data into 80% and 20%, use the mean of 80% data and get the RMSE of 20% test data
    labels = trip.loc[:,'trips']
    train = trip.drop(['trips'], 1)
    X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.2, random_state = 2)
    y_hat = y_test
    y_hat = np.mean(y_train)
    res = rmse(y_hat,y_test)
    return res
train_SJ_label = get_label(trip_SJ)
base_rmse = base_model(train_SJ_label)


In [ ]:
#Feature Engineering - add weather feature
weather = pd.read_csv('../input/weather.csv')
weather.date = pd.to_datetime(weather.date, format='%m/%d/%Y')
weather.loc[weather.events == 'rain', 'events'] = "Rain"
weather.loc[weather.events.isnull(), 'events'] = "Normal"
weather = pd.get_dummies(weather,columns = ["events"])


In [ ]:
def get_weather(weather,city):
    my_dict = {'San Francisco':94107,'Redwood City':94063,'Palo Alto':94301, 'Mountain View':94041, 'San Jose':95113}
    weather_c = weather[weather['zip_code'] == my_dict[city]]
    return weather_c
weather_SJ = get_weather(weather,city)
train_SJ = train_SJ_label.merge(weather_SJ, on = weather_SJ.date)

In [ ]:
#fill in the missing data, dtype transform
def preprocess(train_SJ):
    train_SJ['date'] = train_SJ['date_x']
    train_SJ.drop(['date_y','date_x','zip_code'],1, inplace= True)
    train_SJ.precipitation_inches = pd.to_numeric(train_SJ.precipitation_inches,errors = 'coerce')
    train_SJ.loc[train_SJ.max_gust_speed_mph.isnull(),'max_gust_speed_mph'] = train_SJ.groupby('max_wind_Speed_mph').max_gust_speed_mph.apply(lambda x: x.fillna(x.median()))
    train_SJ.loc[train_SJ.precipitation_inches.isnull(), 
            'precipitation_inches'] = train_SJ[train_SJ.precipitation_inches.notnull()].precipitation_inches.median()    #transform the dtype to numeric
    return train_SJ

In [ ]:
train_SJ = preprocess(train_SJ) 

In [ ]:
####add station
def get_docks(train,station,city):
    stationc = station[station['city'] == city]
    stationc.installation_date = pd.to_datetime(stationc.installation_date, format = "%m/%d/%Y").dt.date
    total_docks = []
    for day in train.date:
        total_docks.append(sum(stationc[stationc.installation_date <= day].dock_count))
    return total_docks


In [ ]:
dock_SJ = get_docks(train_SJ,station,city)
train_SJ['dock'] = dock_SJ

In [ ]:
#Add datetime feature
def add_DateFeature(train):
    calendar = USFederalHolidayCalendar()
    holidays = calendar.holidays(start=train_SJ.date.min(), end=train_SJ.date.max())
    us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    business_days = pd.DatetimeIndex(start=train_SJ.date.min(), end=train_SJ.date.max(), freq=us_bd)
    businessday = pd.to_datetime(business_days, format='%Y/%m/%d').date
    holiday = pd.to_datetime(holidays, format='%Y/%m/%d').date
    train['holiday'] = train.date.isin(holidays)
    train['businessday'] = train.date.isin(businessday)
    train.holiday = train.holiday.map(lambda x : 1 if x == True else 0)
    train.businessday = train.businessday.map(lambda x : 1 if x == True else 0)
    train['year'] = pd.to_datetime(train['date']).dt.year
    train['month'] = pd.to_datetime(train['date']).dt.month
    train['weekday'] = pd.to_datetime(train['date']).dt.weekday
    return train


In [ ]:
train_SJ = add_DateFeature(train_SJ)

In [ ]:
y = train_SJ['trips']
date = train_SJ['date']
train_SJ = train_SJ.drop(['trips','date'],1)


In [ ]:
train_SJ.describe()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_SJ, y, test_size=0.1, random_state = 2)
Lasso = make_pipeline(RobustScaler(),LassoCV(alphas = [1, 0.1, 0.001, 0.0005], random_state = 1))
Lasso.fit(x_train,y_train)
print(x_test.shape)
print(x_train.shape)

In [ ]:
#linear model
n_folds = 5

def linear_model(train,labels):
    x_train, x_test, y_train, y_test = train_test_split(train, labels, test_size=0.2, random_state = 2)
    Lasso = make_pipeline(RobustScaler(),LassoCV(alphas = [1, 0.1, 0.001, 0.0005], random_state = 1))
    Lasso.fit(x_train,y_train)
    Lasso_predict = Lasso.predict(x_test)
    rmse_err = rmse(Lasso_predict,y_test)
    return rmse_err
linear_model(train_SJ,y)

    

In [ ]:
#XGBoost
#XGBoost Model Tuning

def xgb_model(train,labels):
    x_train, x_val, y_val,y_val = train_test_split(train,labels, test_size=0.1, random_state = 2)
    xgb = XGBRegressor(n_estimators = 1000,learning_rate = 0.3,max_depth = 3,eval_metrics = 'rmse',
                       subsample=0.8,
                      gamma = 0.4,
                      min_child_weight = 5)
    xgb.fit(x_train,y_train,early_stopping_rounds = 5,verbose = False)
    xgb_prediction = xgb.predict(x_test)
    err = rmse(y_test,xgb_prediction)
    return err

In [ ]:
#XGBoost
#XGBoost Model Tuning
def xgb_model(train,labels):
    x_train, x_val, y_val,y_val = train_test_split(train,labels, test_size=0.1, random_state = 2)
    xgb = XGBRegressor(n_estimators = 1000,learning_rate = 0.3,max_depth = 3,eval_metrics = 'rmse',
                       subsample=0.8,
                      gamma = 0.4,
                      min_child_weight = 5)
    xgb.fit(x_train,y_train,early_stopping_rounds = 5,verbose = False)
    xgb_prediction = xgb.predict(x_test)
    err = rmse(y_test,xgb_prediction)
    return err

In [ ]:
random_state = 2
xgb = XGBRegressor(n_estimators = 1000,learning_rate = 0.12,max_depth = 6,eval_metrics = 'rmse',
                       subsample=0.8,
                      gamma = 0.1,
                      min_child_weight = 5,
                   seed = random_state)


In [ ]:
def rmsle_cv(model,train_data,y):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train_data, y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)
def scoring(clf):
    scores = np.sqrt(-cross_val_score(clf, x_train, y_train, cv=5, n_jobs=1, scoring = 'neg_mean_squared_error'))
    score = np.mean(scores)
    return scores

In [ ]:
xgb.fit(x_train,y_train)
np.mean(scoring(xgb))


In [ ]:
xgb.feature_importances_

In [ ]:
gbr = GradientBoostingRegressor(learning_rate = 0.12,
                                n_estimators = 150,
                                max_depth = 8,
                                min_samples_leaf = 1,
                                random_state = 2)
scoring(gbr)
gbr.fit(x_train,y_train)


In [ ]:
#feature importance (with tree model)
def plot_importance(model):
    importances = model.feature_importances_
    std = np.std([model.feature_importances_ for feature in model.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1] 
    plt.figure(figsize = (8,5))
    plt.title("Feature importances")
    plt.bar(range(x_train.shape[1]), importances[indices], color="r", align="center")
    plt.xticks(range(x_train.shape[1]), indices)
    plt.xlim([-1, x_train.shape[1]])
    plt.show()
i = 0
for feature in x_train:
    print (i, feature)
    i += 1
plot_importance(gbr)
    

In [ ]:
#feature importance(with Lasso)
plot_importance(gbr)

In [ ]:
#feature selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import LinearSVC
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
def var_feature_sel(X):
    sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
    sel.fit_transform(X)
    return X

def mod_feature_sel(X,y):
    lasso = Lasso(alpha = 0.01).fit(X, y)
    print (lasso.feature_importance_)
    model = SelectFromModel(lasso, prefit=True)
    X_new = model.transform(X)
    return X_new

In [ ]:
#data 
ntrain = x_train.shape[0]
ntest = x_test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

In [ ]:
##helper function
class SklearnHelper(object):
    def __init__(self, clf, seed = 0, params = None):
        params['random_state'] = seed
        self.clf = clf(**params)
    def train(self, x_train,y_train):
        self.clf.fit(x_train,y_train)
    def predict(self,x):
        return self.clf.predict(x)
    def fit(self, x, y):
        return self.clf.fit(x,y)
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
        


In [ ]:
#when predicting the test datasets, we need to get the results predicted by first layer. But if we use the test sets, then it might
#cause overfitting when fitting new model of second layer.
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [ ]:
#def parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}
# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}


In [ ]:
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
x_trainv = x_train.values #values 就是说从dataframe转化成matrix
y_trainv = y_train.ravel()
x_testv = x_test.values



In [ ]:
#rf_oof_train, rf_oof_test = get_oof(rf,x_trainv, y_trainv, x_testv) # Random Forest
ada_oof_train, ada_oof_test = get_oof(ada, x_trainv, y_trainv, x_testv) # AdaBoost 
gb_oof_train, gb_oof_test = get_oof(gb,x_trainv, y_trainv, x_testv) # Gradient Boost

In [ ]:
ada_feature = ada.feature_importances(x_train, y_train)
gb_feature = gb.feature_importances(x_train,y_train)

In [ ]:
#create a dataframe for the output
cols = x_train.columns.values
# Create a dataframe with features


In [ ]:
#second level
base_predictions_train = pd.DataFrame( {
     'AdaBoost': ada_oof_train.ravel(),
      'GradientBoost': gb_oof_train.ravel()
    })
base_predictions_train.head()

In [ ]:
data = [
    go.Heatmap(
        z= base_predictions_train.astype(float).corr().values ,
        x=base_predictions_train.columns.values,
        y= base_predictions_train.columns.values,
          colorscale='Viridis',
            showscale=True,
            reversescale = True
    )
]
py.iplot(data, filename='labelled-heatmap')

In [ ]:
x_train = np.concatenate(( ada_oof_train, gb_oof_train), axis=1)
x_test = np.concatenate((ada_oof_test, gb_oof_test), axis=1)

In [ ]:
 gbm = XGBRegressor(n_estimators = 2000,learning_rate = 0.01,max_depth = 6,eval_metrics = 'rmse',
                       subsample=0.9,
                      gamma = 0.1,
                      min_child_weight = 6,
                   seed = random_state).fit(x_train, y_train)
predictions = gbm.predict(x_test)
rmse(predictions,y_test)